### PromptTemplate
* PromptTemplate
* ChatPromptTemplate
* ChatMessagePromptTemplate
* FewShotPromptTemplate
* PartialPrompt

In [ ]:
#%pip install -q langchain langchain-openai

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)
chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 대규모 언어 모델이며, 수십억 개의 매개변수를 학습하여 자연어 처리 작업을 수행합니다. ChatGPT의 학습에는 레이블이 '
 '지정된 대규모 데이터 세트와 최적화 알고리즘을 사용한 지도 학습 및 자체 학습이 포함됩니다. 이 모델은 주어진 입력에 대해 다음에 오는 '
 '단어를 예측하도록 학습하며, 이 학습을 통해 자연어 대화에 참여할 수 있는 능력을 습득합니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요." 

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
# 기존 템플릿에 결합
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)

combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})

pprint(response)

('ChatGPT 모델의 학습 원리는 다음과 같습니다.\n'
 '\n'
 'ChatGPT는 대규모 언어 데이터셋을 기반으로 하는 딥러닝 모델입니다. 이 모델은 주어진 문맥에 따라 다음에 올 단어를 예측하도록 '
 '학습되며, 이를 위해 방대한 텍스트 데이터를 스스로 학습합니다. 이러한 학습 과정을 통해 ChatGPT는 자연스러운 대화 흐름을 이해하고 '
 '생성할 수 있는 능력을 습득합니다.\n'
 '\n'
 'ChatGPT 모델의 장점은 다음과 같습니다.\n'
 '\n'
 '*   자연스러운 대화 생성: ChatGPT는 사람처럼 자연스러운 대화 흐름을 생성할 수 있습니다.\n'
 '*   지식 정보 제공: ChatGPT는 방대한 텍스트 데이터로부터 학습했기 때문에, 다양한 주제에 대한 정보를 제공할 수 있습니다.\n'
 '*   창의적 콘텐츠 생성: ChatGPT는 창의적인 콘텐츠, 예를 들어 이야기나 시 등을 생성할 수 있습니다.\n'
 '\n'
 'ChatGPT 모델과 비슷한 AI 모델은 다음과 있습니다.\n'
 '\n'
 '*   **LLaMA**: 메타에서 개발한 대규모 언어 모델입니다.\n'
 '*   **PaLM**: 구글이 개발한 대규모 언어 모델로, 다양한 자연어 처리 작업에서 뛰어난 성능을 발휘합니다.\n'
 '*   **Bard**: 구글이 개발한 AI 챗봇으로, Gemini 모델을 기반으로 합니다.')


In [9]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt))
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.']
<class 'str'>
('GPT-4 모델은 대규모 데이터 세트에 대해 훈련된 생성형 언어 모델로서, 주어진 입력에 대해 다음에 올 가능성이 높은 단어를 예측하도록 '
 '설계되었습니다. 이 모델은 강화 학습 및 지도 학습을 사용하여 미세 조정되었으며 인터넷, 책 및 대화 데이터로 훈련되었습니다.')
<class 'str'>
('Gemma는 컴퓨터가 자연어 텍스트를 생성하도록 훈련된 인공지능(AI) 모델입니다. 텍스트 데이터 세트에 대해 훈련되었으며 주어진 '
 '프롬프트나 입력에 따라 일관되고 관련성이 높은 텍스트를 생성하는 방법을 학습했습니다. 기본적으로 Gemma는 다음 단어, 구절 또는 '
 '단락을 예측하여 텍스트를 생성하며, 이 예측은 학습한 패턴과 언어 구조를 기반으로 합니다.')


##### 2) ChatPromptTemplate
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 튜플 형식 system, user, assistant 메시지 지원
* 간결성과 가독성이 높고 단순한 구조

In [10]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
messages

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]

In [11]:
# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델입니다. 이 모델은 대규모 언어 모델(LLM, Large Language Model) 기반의 인공지능으로, 자연어 처리 분야에서 우수한 성능을 발휘합니다. ChatGPT의 학습 원리는 다음과 같습니다.

1. **대규모 데이터 수집**: ChatGPT는 인터넷에서 수집한 대규모 텍스트 데이터를 기반으로 학습합니다. 이 데이터에는 책, 기사, 웹사이트 등 다양한 출처의 텍스트가 포함되어 있습니다.

2. **토큰화**: 수집된 텍스트 데이터는 토큰이라는 단위로 나누어집니다. 토큰은 단어, 구절 또는 기호가 될 수 있습니다. 예를 들어, 문장 "안녕하세요, ChatGPT입니다."는 ["안녕하세요", ",", "ChatGPT", "입니다", "."]과 같이 토큰화될 수 있습니다.

3. **자기 지도 학습(Self-Supervised Learning)**: ChatGPT는 자기 지도 학습 방식을 사용하여 학습합니다. 이 방식에서는 모델이 주어진 토큰 시퀀스에서 다음 토큰을 예측하도록 훈련합니다. 즉, 모델은 문맥을 이해하고 다음에 올 토큰을 예측하는 방식으로 학습합니다.

4. **트랜스포머 아키텍처**: ChatGPT는 트랜스포머 아키텍처를 기반으로 합니다. 트랜스포머는 셀프 어텐션 메커니즘을 사용하여 입력 시퀀스의 모든 토큰 간의 관계를 모델링합니다. 이 아키텍처는 병렬화가 가능하여 대규모 데이터에 효율적으로 학습할 수 있습니다.

5. **마스킹된 언어 모델링(Masked Language Modeling)**: ChatGPT의 학습에는 마스킹된 언어 모델링이라는 기술이 사용됩니다. 이 기술에서는 입력 토큰 시퀀스의 일부 토큰을 무작위로 마스킹하고, 모델이 마스킹된 토큰을 예측하도록 합니다. 이를 통해 모델은 토큰 간의 관계를 더 잘 이해하게 됩니다.

6. **다음 토큰 예측**: 학습 과정에서 모델은 주어진 토큰 시퀀

In [12]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
ChatGPT는 OpenAI에서 개발한 대화형 AI 모델입니다. 이 모델은 대규모의 텍스트 데이터를 학습하여 자연어 처리 및 생성 능력을 갖추게 되었습니다. ChatGPT의 학습 원리는 다음과 같습니다.

1. **대규모 데이터 수집**: ChatGPT는 인터넷에서 수집한 대규모의 텍스트 데이터를 학습합니다. 이 데이터에는 책, 기사, 웹사이트 등 다양한 출처의 텍스트가 포함됩니다.

2. **토큰화**: 수집된 텍스트 데이터는 토큰이라는 작은 단위로 나누어집니다. 토큰은 단어, 구절, 심지어 개별 문자일 수 있습니다.

3. **자기 지도 학습**: ChatGPT는 자기 지도 학습(self-supervised learning) 방식을 사용합니다. 이 방식에서는 모델이 주어진 토큰 시퀀스에서 다음 토큰을 예측하도록 학습합니다. 즉, 모델은 문맥을 이해하고 다음에 올 토큰을 예측하는 방식으로 학습합니다.

4. **트랜스포머 아키텍처**: ChatGPT는 트랜스포머 아키텍처(transformer architecture)를 기반으로 합니다. 이 아키텍처는 입력 토큰 시퀀스를 처리하고, 각 토큰 간의 관계를 모델링하여 출력 토큰 시퀀스를 생성합니다.

5. **마스킹된 언어 모델링**: ChatGPT의 학습에는 마스킹된 언어 모델링(masked language modeling)이라는 기술이 사용됩니다. 이 기술에서는 입력 토큰 시퀀스의 일부 토큰을 무작위로 마스킹하고, 모델이 마스킹된 토큰을 예측하도록 합니다. 이를 통해 모델은 토큰 간의 관계를 더 잘 이해하게 됩니다.

6. **다중 과제 학습**: ChatGPT는 다중 과제 학습(multi-task learning) 방식을 사용합니다. 이 방식에서는 모델이 여러 관련 과제를 동시에 학습하여 성능을 향상시킵니다. ChatGPT는 대화 생성, 질문 답변, 텍스트 요약 등 다양한 과제를 학습합니다.

7. **강화 학습**: ChatGPT는 강화 학습(reinforcement learning

#### 3) ChatMessagePromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [13]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

 AI systems can perform tasks that normally require human intelligence, such as recognizing speech, interpreting languages, and making decisions. A subset of AI is machine learning (ML), which refers to the ability of systems to automatically learn from data, without being explicitly programmed. A subset of machine learning is deep learning, which uses algorithms based on the structure and function of the human brain. 
Deep learning uses artificial neural networks to analyze data, which are composed of multiple layers of interconnected nodes or "neurons." These networks are capable of learning complex patterns in data and making accurate predictions or decisions. Deep learning models can be trained on large datasets and can learn to recognize images, speech, text, and other types of data. 
Some common applications of deep learning include image and speech recognition, natural language processing, and autonomous vehicles. Deep learning models have achieved state-of-the-art performance i

##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

In [ ]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")
# 일반적인 방법
llm = ChatOpenAI(
    b ase_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")
#result = llm.invoke("양자 얽힘이 무엇인가요?")
print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계의 행성은 총 8개입니다. 

1.  수성: 태양과 가장 가까운 행성으로, 표면이 암석으로 구성되어 있습니다.
2.  금성: 태양계에서 두 번째로 가까운 행성으로, 두꺼운 대기로 인해 극심한 온실 효과가 있습니다.
3.  지구: 생명체가 존재하는 유일한 행성으로, 대기와 물이 있어 생명 유지에 적합합니다.
4.  화성: 태양계에서 네 번째로 가까운 행성으로, 표면이 붉은색이며 과학자들이 생명체 존재 가능성을 연구하고 있습니다.
5.  목성: 태양계에서 가장 큰 행성으로, 가스 행성입니다. 강력한 자기장과 수많은 위성을 가지고 있습니다.
6.  토성: 태양계에서 두 번째로 큰 행성으로, 가스 행성입니다. 아름다운 고리를 가지고 있습니다.
7.  천왕성: 태양계에서 일곱 번째로 가까운 행성으로, 가스 행성입니다. 자전축이 기울어져 있어 극단적인 기후 변화를 겪습니다.
8.  해왕성: 태양계에서 가장 먼 행성으로, 가스 행성입니다. 강력한 바람과 깊은 바다를 가지고 있습니다.


In [5]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
# gpt-3.5-turbo
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
print(result.content)

result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

result = chain.invoke({"input": "아르곤이 무엇인가요?"})
print(result.content)

result = chain.invoke({"input": "화성의 대기 구성 요소를 알려주세요."})
print(result.content)

### 태양계의 행성
1. **수성** - 태양과 가장 가까운 행성으로, 매우 작고 온도가 극심하게 변합니다.
2. **금성** - 밝고 뜨거운 행성으로, 강한 온실 효과로 표면이 매우 뜨겁습니다.
3. **지구** - 생명체가 살고 있는 유일한 행성으로, 대기 중 산소가 풍부합니다.
4. **화성** - 붉은 행성으로, 과거에는 물이 있었다고 추정되며 미래의 인간 탐사 후보지입니다.
5. **목성** - 태양계에서 가장 큰 행성으로, 가스 거성입니다.
6. **토성** - 아름다운 고리를 가진 가스 거성입니다.
7. **천왕성** - 자전축이 기울어져 있어 극단적인 계절 변화를 겪습니다.
8. **해왕성** - 가장 멀리 있는 행성으로, 매우 춥고 강한 바람이 불습니다.
### 양자 얽힘
- **정의**: 두 개 이상의 입자가 서로 연결되어 있어, 한 입자의 상태가 다른 입자의 상태에 영향을 미치는 현상입니다.
- **예시**: 두 입자가 얽혀 있을 때, 한 입자의 스핀을 측정하면 다른 입자의 스핀도 즉시 결정됩니다.
- **특징**: 공간적으로 멀리 떨어져 있어도 즉시 정보가 전달됩니다.
### 아르곤
아르곤은 **원자번호 18번**인 화학 원소로, **비활성 기체**입니다. 

- **성질**: 
  - 무색, 무취, 무독성입니다.
  - 반응성이 매우 낮아 화학적으로 안정적입니다.

- **용도**: 
  - 전구(백열등, 네온사인 등)에 사용됩니다.
  - 반도체 제조 공정에서 사용됩니다.
  - 레이저 기술 등에 활용됩니다.
### 화성의 대기 구성
- **이산화탄소 (95.3%)**: 대기의 대부분을 차지합니다.
- **질소 (2.7%)**: 상대적으로 적은 양이 존재합니다.
- **아르곤 (1.6%)**: 화성의 대기에서 발견되는 기체입니다.
- **산소와 물 (적은 양)**: 화성의 대기는 매우 얇고, 극소량의 산소와 물이 존재합니다.


#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [6]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = model.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 여름에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 여름에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 여름에 주로 발생하는 지구과학 현상은 다음과 같습니다.

1.  **태풍 발생**: 태풍은 열대 저기압의 일종으로, 여름에 주로 발생합니다. 태풍은 강한 바람과 비를 동반하며, 큰 피해를 입힐 수 있습니다.
2.  **장마**: 장마는 여름에 발생하는 현상으로, 일정 기간 동안 비가 계속 내리는 현상입니다. 장마는 한국, 중국, 일본 등 동아시아 지역에서 주로 발생합니다.
3.  **엘니뇨**: 엘니뇨는 남미 에콰도르와 페루 연근해의 수온이 상승하는 현상으로, 여름에 주로 발생합니다. 엘니뇨는 전 세계의 날씨에 영향을 미치며, 가뭄이나 홍수 등 극심한 날씨 변화를 일으킬 수 있습니다.

이러한 현상들은 여름에 지구과학적으로 중요한 사건들입니다.


In [9]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
#season = get_current_season("south")  # 계절 값 얻기
season = get_current_season()
print(f"현재 계절: {season}")

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.0
)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x: season}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season}에 발생하는 자연 현상:\n{response}")

현재 계절: 여름

 여름에 발생하는 자연 현상:
1.  **장마**: 장마는 한국, 중국, 일본 등 동아시아 지역에서 여름에 발생하는 현상으로, 남쪽에서 북쪽으로 이동하는 저기압과 고기압의 영향으로 발생합니다. 장마는 일반적으로 6월 중순부터 7월 중순까지 지속되며, 이 기간 동안 많은 비가 내립니다. 장마는 농작물에 큰 영향을 미치며, 홍수나 산사태를 유발할 수 있습니다.
2.  **태풍**: 태풍은 열대 지방에서 발생하는 강력한 저기압으로, 강한 바람과 많은 비를 동반합니다. 태풍은 여름에 주로 발생하며, 한국, 일본, 필리핀 등 동아시아 지역에서 큰 피해를 입힐 수 있습니다. 태풍은 강한 바람과 폭우로 인해 홍수, 산사태, 건물 피해 등을 일으킬 수 있습니다.
3.  **열대야**: 열대야는 밤에도 기온이 25도 이상으로 유지되는 현상으로, 여름에 주로 발생합니다. 열대야는 도시 지역에서 더 심해지며, 사람들의 수면과 건강에 영향을 미칠 수 있습니다. 열대야는 열사병, 탈진, 수면 장애 등을 유발할 수 있습니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [15]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 200만원 환율을 알려주고, 보통 팁은 얼마나 줘야되 .. ?",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정 (GPT-4o-mini 사용)
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# ✅ 모델에 프롬프트 전달 및 응답 받기
response = model.invoke(prompt.format())

# ✅ 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1362.35원 기준으로 환율 정보를 알려드립니다. 200만원 환율을 알려주고, 보통 팁은 얼마나 줘야되 .. ?
 모델 응답: 현재 환율 기준으로 200만원을 달러로 환산하면:

200만원 ÷ 1362.35원/달러 ≈ 1467.41달러

가 됩니다.

팁은 상황에 따라 다르지만, 일반적으로 서비스의 질과 상황에 따라 5-15% 정도의 팁을 주는 것이 관례입니다. 예를 들어:

* 식당에서 식사할 경우: 10-15% (200만원 * 0.1 / 1362.35 ≈ 14.67달러, 200만원 * 0.15 / 1362.35 ≈ 22달러)
* 택시 기사에게 팁: 5-10% (200만원 * 0.05 / 1362.35 ≈ 7.33달러, 200만원 * 0.1 / 1362.35 ≈ 14.67달러)

하지만 이는 어디까지나 일반적인 예시이며, 상황에 따라 팁의 금액과 비율이 달라질 수 있습니다.
